In [ ]:
"""
Parser całej ustawy w PDF → strukturalny JSON
Wersja do użycia bezpośrednio w jednej komórce Colab / notebooka.
Po wczytaniu funkcji wywołaj np.:

    process_act("/content/raw.pdf", "/content/ustawa_processed.json")

wtedy w `/content/` znajdziesz gotowy plik JSON.
"""

import re
import json
from pathlib import Path

try:
    import pdfplumber  # lightweight PDF text extractor
except ImportError as e:
    raise ImportError("Zainstaluj pdfplumber: pip install pdfplumber") from e

###########################
#  META‑DANE (do edycji)  #
###########################
META = {
    "document_title": (
        "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 5 lutego 2025 r. "
        "w sprawie ogłoszenia jednolitego tekstu ustawy o ubezpieczeniu społecznym rolników"
    ),
    "date": "2025-02-14",  # data ogłoszenia
    "item": "197",        # pozycja w Dz.U.
}

###########################
#  REGEX + SKRÓTY         #
###########################
ABBREVIATIONS = {
    r"\bDz\. U\.": "Dziennik Ustaw",
    r"\b(późn\.|z późn\.) zm\.": "z późniejszymi zmianami",
    r"\bust\.": "ustęp",
    r"\bart\.": "artykuł",
}

CHAPTER_RE = re.compile(r"^Rozdział\s+(?P<num>[0-9A-Za-z]+)\s*(?P<title>.*)")
ARTICLE_RE = re.compile(r"Art\.\s*(?P<num>[0-9A-Za-z]+)\.")
SUBSECTION_RE = re.compile(r"^(?P<num>[0-9]+[a-z]?)\)\s+|^(?P<num_dot>[0-9]+[a-z]?\.)\s+", re.MULTILINE)

###########################
#  FUNKCJE POMOCNICZE     #
###########################

def clean_text(text: str) -> str:
    """Usuwa nadmiar białych znaków i rozwija skróty."""
    text = re.sub(r"\s+", " ", text.strip())
    for abbr, full in ABBREVIATIONS.items():
        text = re.sub(abbr, full, text, flags=re.IGNORECASE)
    return text


def load_pdf_text(pdf_path: Path) -> str:
    """Łączy tekst ze wszystkich stron PDF‑a w jeden łańcuch."""
    with pdfplumber.open(str(pdf_path)) as pdf:
        pages = [p.extract_text() or "" for p in pdf.pages]
    return "\n".join(pages)


def split_chapters(lines):
    """Generator zwracający (chapter_header_match, chapter_lines)."""
    current_header = None
    buffer = []
    for line in lines:
        ch_match = CHAPTER_RE.match(line)
        if ch_match:
            if current_header is not None:
                yield current_header, buffer
            current_header = ch_match
            buffer = []
        else:
            buffer.append(line)
    if current_header is not None:
        yield current_header, buffer


def parse_subsections(article_body: str):
    """Zwraca listę podpunktów dla jednego artykułu."""
    subsections = []
    body = re.sub(r"\n-\s+", "\n", article_body)
    matches = list(SUBSECTION_RE.finditer(body))
    if not matches:
        content = clean_text(body)
        if content:
            subsections.append({"number": "1", "content": content})
        return subsections

    for idx, m in enumerate(matches):
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(body)
        num = (m.group("num") or m.group("num_dot") or "").replace(")", "").replace(".", "").strip()
        content = clean_text(body[start:end])
        if not content or content.lower().startswith("(uchylony)"):
            content = "Treść uchylona"
        subsections.append({"number": num, "content": content})
    return subsections


def parse_articles(chapter_lines):
    """Zwraca listę artykułów z podpunktami dla danego rozdziału."""
    joined = "\n".join(chapter_lines)
    matches = list(ARTICLE_RE.finditer(joined))
    articles = []
    for idx, m in enumerate(matches):
        art_num = m.group("num")
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(joined)
        body = joined[start:end]
        subsections = parse_subsections(body)
        articles.append({"article": f"{art_num}.", "subsections": subsections})
    return articles

###########################
#  GŁÓWNA FUNKCJA         #
###########################

def process_act(pdf_path: str, output_json: str = "ustawa_processed.json"):
    """Konwertuje cały PDF ustawy do strukturalnego JSON‑a."""
    raw_text = load_pdf_text(Path(pdf_path))
    lines = [l for l in raw_text.splitlines() if l.strip()]

    chapters_json = []
    for header_match, chap_lines in split_chapters(lines):
        chap_num = header_match.group("num").rstrip(".")
        chap_title = header_match.group("title").strip()
        articles = parse_articles(chap_lines)
        chapters_json.append({
            "number": chap_num,
            "title": chap_title,
            "articles": articles,
        })

    output = {
        "document": {
            "title": META["document_title"],
            "date": META["date"],
            "item": META["item"],
            "chapters": chapters_json,
        }
    }

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)
    print(f"✔️  Zapisano wynik do {output_json}. Rozdziałów: {len(chapters_json)} | Artykułów: {sum(len(c['articles']) for c in chapters_json)}")